In [1]:
import ipywidgets as widgets
from IPython.utils import io
from IPython.display import display
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time

In [2]:
from rdkit import Chem

from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
from random import randrange
import svgutils.compose as sc


In [3]:
import argparse
import os.path
import sys
from datetime import timedelta
from pathlib import Path
from typing import Type, Optional, Sequence, List, Iterable, Callable
#sys.path.insert(0, os.getcwd())
sys.path.append( os.getcwd())


import numpy as np
import pandas as pd
from random import randrange

from golem_examples.molecule_search.analysis import get_final_dataset, visualize_results, get_statistical_significance,plot_experiment_comparison
from golem.core.dag.verification_rules import has_no_self_cycled_nodes, has_no_isolated_components, \
    has_no_isolated_nodes
from golem.core.optimisers.adaptive.operator_agent import MutationAgentTypeEnum
from golem.core.optimisers.genetic.gp_optimizer import EvoGraphOptimizer
from golem.core.optimisers.genetic.gp_params import GPAlgorithmParameters
from golem.core.optimisers.genetic.operators.crossover import CrossoverTypesEnum
from golem.core.optimisers.genetic.operators.elitism import ElitismTypesEnum
from golem.core.optimisers.genetic.operators.inheritance import GeneticSchemeTypesEnum
from golem.core.optimisers.objective import Objective
from golem.core.optimisers.optimizer import GraphGenerationParams, GraphOptimizer
from golem.core.optimisers.adaptive.agent_trainer import AgentTrainer
from golem.core.optimisers.adaptive.history_collector import HistoryReader
from rdkit import RDConfig
from rdkit.Chem.rdchem import BondType

from golem_examples.molecule_search.mol_adapter import MolAdapter
from golem_examples.molecule_search.mol_advisor import MolChangeAdvisor
from golem_examples.molecule_search.mol_graph import MolGraph
from golem_examples.molecule_search.mol_graph_parameters import MolGraphRequirements
from golem_examples.molecule_search.mol_metrics import CocrystalsMetrics, sa_score
from golem_examples.molecule_search.mol_mutations import CHEMICAL_MUTATIONS

sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))

def moltosvg(mol, molSize = (300,300), kekulize = True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return svg.replace('svg:','')

def get_methane() -> MolGraph:
    methane = 'C'
    return MolGraph.from_smiles(methane)


def pretrain_agent(optimizer: EvoGraphOptimizer, objective: Objective, results_dir: str) -> AgentTrainer:
    agent = optimizer.mutation.agent
    trainer = AgentTrainer(objective, optimizer.mutation, agent)
    # load histories
    history_reader = HistoryReader(Path(results_dir))
    # train agent
    trainer.fit(histories=history_reader.load_histories(), validate_each=10)
    return trainer


def molecule_search_setup(optimizer_cls: Type[GraphOptimizer] = EvoGraphOptimizer,
                          adaptive_kind: MutationAgentTypeEnum = MutationAgentTypeEnum.random,
                          max_heavy_atoms: int = 50,
                          atom_types: Optional[List[str]] = None,
                          bond_types: Sequence[BondType] = (BondType.SINGLE, BondType.DOUBLE, BondType.TRIPLE),
                          timeout: Optional[timedelta] = None,
                          num_iterations: Optional[int] = None,
                          pop_size: int = 20,
                          drug='CN1C2=C(C(=O)N(C1=O)C)NC=N2',
                          initial_molecules: Optional[Sequence[MolGraph]] = None):
    requirements = MolGraphRequirements(
        max_heavy_atoms=max_heavy_atoms,
        available_atom_types=atom_types or ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I'],
        bond_types=bond_types,
        early_stopping_timeout=np.inf,
        early_stopping_iterations=np.inf,
        keep_n_best=4,
        timeout=timeout,
        num_of_generations=num_iterations,
        keep_history=True,
        n_jobs=-1,
        history_dir=None,
    )
    gp_params = GPAlgorithmParameters(
        pop_size=pop_size,
        max_pop_size=pop_size,
        multi_objective=True,
        genetic_scheme_type=GeneticSchemeTypesEnum.steady_state,
        elitism_type=ElitismTypesEnum.replace_worst,
        mutation_types=CHEMICAL_MUTATIONS,
        crossover_types=[CrossoverTypesEnum.none],
        adaptive_mutation_type=adaptive_kind
    )
    graph_gen_params = GraphGenerationParams(
        adapter=MolAdapter(),
        rules_for_constraint=[has_no_self_cycled_nodes, has_no_isolated_components, has_no_isolated_nodes],
        advisor=MolChangeAdvisor(),
    )

    metrics = CocrystalsMetrics(drug)
    objective = Objective(
        quality_metrics={'orthogonal_planes': metrics.orthogonal_planes,
                         'unobstructed': metrics.unobstructed,
                         'h_bond_bridging': metrics.h_bond_bridging,
                         'sa_score': sa_score},
        is_multi_objective=True
    )

    initial_graphs = initial_molecules or [get_methane()]
    initial_graphs = graph_gen_params.adapter.adapt(initial_graphs)

    # Build the optimizer
    optimiser = optimizer_cls(objective, initial_graphs, requirements, graph_gen_params, gp_params)
    return optimiser, objective


def run_experiment(optimizer_setup: Callable,
                   optimizer_cls: Type[GraphOptimizer] = EvoGraphOptimizer,
                   adaptive_kind: MutationAgentTypeEnum = MutationAgentTypeEnum.random,
                   max_heavy_atoms: int = 50,
                   atom_types: Optional[List[str]] = None,
                   bond_types: Sequence[BondType] = (BondType.SINGLE, BondType.DOUBLE, BondType.TRIPLE),
                   initial_data_path: Optional[str] = None,
                   col_name: str = 'generated_coformers',
                   pop_size: int = 20,
                   num_trials: int = 1,
                   trial_timeout: Optional[int] = None,
                   trial_iterations: Optional[int] = None,
                   visualize: bool = False,
                   save_history: bool = True,
                   drug='CN1C2=C(C(=O)N(C1=O)C)NC=N2',
                   result_dir: Optional[str] = None,
                   pretrain_dir: Optional[str] = None,
                   indx:int=0
                   ):
    optimizer_id = optimizer_cls.__name__.lower()[:3]
    experiment_id = f'Experiment [optimizer={optimizer_id} pop_size={pop_size}]'
    exp_name = f'init_{Path(initial_data_path).stem}_{optimizer_id}_{adaptive_kind.value}_popsize{pop_size}_min{trial_timeout}'
    result_dir = Path(result_dir) / exp_name

    initial_smiles = pd.read_csv(initial_data_path)[col_name]


    initial_molecules = []
    initial_smiles = initial_smiles[indx:indx+1]
    for smiles in initial_smiles:
        try:
            mol = MolGraph.from_smiles(smiles)
            initial_molecules.append(mol)
        except Exception as ex:
            print(ex)
            continue

    trial_results = []
    trial_histories = []
    trial_timedelta = timedelta(minutes=trial_timeout) if trial_timeout else None
    print(exp_name)
    for trial in range(num_trials):
        optimizer, objective = optimizer_setup(optimizer_cls,
                                               adaptive_kind,
                                               max_heavy_atoms,
                                               atom_types,
                                               bond_types,
                                               trial_timedelta,
                                               trial_iterations,
                                               pop_size,
                                               drug,
                                               initial_molecules)
        if pretrain_dir:
            pretrain_agent(optimizer, objective, pretrain_dir)

        found_graphs = optimizer.optimise(objective)
        history = optimizer.history
        if save_history:
            result_dir.mkdir(parents=True, exist_ok=True)
            history.save(result_dir / f'history_trial_{trial}.json')
        trial_results.extend(history.final_choices)
        trial_histories.append(history)

    # Compute mean & std for metrics of trials
    ff = objective.format_fitness
    trial_metrics = np.array([ind.fitness.values for ind in trial_results])
    trial_metrics_mean = trial_metrics.mean(axis=0)
    trial_metrics_std = trial_metrics.std(axis=0)
    print(f'Experiment {experiment_id}\n'
          f'finished with metrics:\n'
          f'mean={ff(trial_metrics_mean)}\n'
          f' std={ff(trial_metrics_std)}')
    return exp_name, objective


# def parameter():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-drug',required=False, type=str, default='CN1C2=C(C(=O)N(C1=O)C)NC=N2')
#     parser.add_argument('-data_paths',required=False, nargs='+',
#                         default=r'results\CVAE_all_valid.csv',
#                         help='')
#     parser.add_argument('-coformers_col_name',required=False, type=str, default='generated_coformers')
#     parser.add_argument('-results_folder',required=False, type=str, default='results')
#     parser.add_argument("-f", required=False, default='results')
#     args = parser.parse_args()
#     return args


Choose drug that need to find co-crystal for

In [44]:
theophylline_smiles = pd.read_csv(r'results\CVAE_all_valid.csv')
benzamide_smiles = pd.read_csv(r'results\NC(=O)C1=CC=CC=C1.csv')
acetazolamide_smiles = pd.read_csv(r'results\CC(=O)Nc1nnc(s1)S(N)(=O)=O.csv')
caffeine_smiles = pd.read_csv(r'results\CN1C=NC2=C1C(=O)N(C)C(=O)N2C.csv')
pyrazinecarboxamide_smiles = pd.read_csv(r'results\NC(=O)c1cnccn1.csv')

In [48]:
drugs = widgets.Dropdown(
    options=['Theophylline', 'Benzamide', 'Acetazolamide', 'Caffeine','Pyrazinecarboxamide'],
    value='Theophylline',
    description='Choose drug:',
    disabled=False
)
display(drugs)
draw_molecules = widgets.interact.options(manual=True, manual_name="Show drug")
drugs_list = {'Theophylline':['CN1C2=C(C(=O)N(C1=O)C)NC=N2',theophylline_smiles['generated_coformers'],theophylline_smiles['sa_score']],
              'Benzamide':['NC(=O)C1=CC=CC=C1',benzamide_smiles['generated_coformers'],benzamide_smiles['sa_score']],
              'Acetazolamide':['CC(=O)Nc1nnc(s1)S(N)(=O)=O',acetazolamide_smiles['generated_coformers'],acetazolamide_smiles['sa_score']],
              'Caffeine':['CN1C=NC2=C1C(=O)N(C)C(=O)N2C',caffeine_smiles['generated_coformers'],caffeine_smiles['sa_score']],
              'Pyrazinecarboxamide':['NC(=O)c1cnccn1',pyrazinecarboxamide_smiles['generated_coformers'],pyrazinecarboxamide_smiles['sa_score']]}
@draw_molecules
def generate():
    # display((drugs_list[drugs.value][0]))
    # display(SVG(moltosvg(Chem.MolFromSmiles(drugs_list[drugs.value][0]))))

    display(Draw.MolsToGridImage([Chem.MolFromSmiles(drugs_list[drugs.value][0])],
                                    molsPerRow=min(1, 1),
                                    legends=[drugs_list[drugs.value][0]],
                                    subImgSize=(250, 250)))

Dropdown(description='Choose drug:', options=('Theophylline', 'Benzamide', 'Acetazolamide', 'Caffeine', 'Pyraz…

interactive(children=(Button(description='Show drug', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

In [6]:
# pop_size = widgets.Dropdown(
#     options=[5, 10, 20, 50],
#     value=10,
#     description='Population size:',
#     disabled=False
# )
# trial_iterations = widgets.Dropdown(
#     options=[5, 10, 20, 50],
#     value=20,
#     description='Iterations:',
#     disabled=False
# )
# display(pop_size)
# display(trial_iterations)

In [17]:
pop_size = widgets.IntSlider(
    value=10,
    min=5,
    max=25,
    step=1,
    description='Population size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
trial_iterations = widgets.IntSlider(
    value=20,
    min=10,
    max=55,
    step=1,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)



In [ ]:
+"\nSA  :"+str(drugs_list[drugs.value][2])

In [72]:
generate_molecules = widgets.interact.options(manual=True, manual_name="Generate")

@generate_molecules
def generate():
    global ind
    ind = randrange(len(drugs_list[drugs.value][1]))
    for i in range(randrange(5)):
        print('Generation in progress...')
        time.sleep(0.5)
    print('Generation done!')
    display(Draw.MolsToGridImage([Chem.MolFromSmiles(drugs_list[drugs.value][0]),Chem.MolFromSmiles(drugs_list[drugs.value][1][ind])],
                                    molsPerRow=min(2, 2),
                                    legends=[drugs_list[drugs.value][0],str(drugs_list[drugs.value][1][ind]+'  | SA score:  '+str(round(drugs_list[drugs.value][2][ind],2)))],
                                    subImgSize=(500, 500)))
    #display(SVG(moltosvg(Chem.MolFromSmiles(drugs_list[drugs.value][1][ind]))))
    

interactive(children=(Button(description='Generate', style=ButtonStyle()), Output()), _dom_classes=('widget-in…

In [19]:
display(pop_size)
display(trial_iterations)

IntSlider(value=5, continuous_update=False, description='Population size:', max=25, min=5)

IntSlider(value=55, continuous_update=False, description='Iterations:', max=55, min=10)

In [10]:
from golem.core.log import Log
def main(indx):
    Log().reset_logging_level(100)
    #args = parameter()
    drug = drugs_list[drugs.value][0]
    data_paths = [r'results\CVAE_all_valid.csv']
    col_name = 'generated_coformers'
    results_folder = 'results'

    exp_ids = []
    for init_path in data_paths:
        exp_id, objective = run_experiment(molecule_search_setup,
                                           initial_data_path=init_path,
                                           col_name=col_name,
                                           max_heavy_atoms=50,
                                           trial_timeout=10,
                                           trial_iterations=trial_iterations.value,
                                           pop_size=pop_size.value,
                                           visualize=False,
                                           num_trials=1,
                                           drug=drug,
                                           result_dir=results_folder,
                                           adaptive_kind=MutationAgentTypeEnum.random,
                                           pretrain_dir=None,
                                           indx = indx
                                           )
        exp_ids.append(exp_id)
        exp_results_folder = os.path.join(results_folder, exp_id)
        os.makedirs(exp_results_folder, exist_ok=True)

        init_df = pd.read_csv(init_path)
        golem_result = get_final_dataset(exp_results_folder)
    return exp_results_folder

In [11]:


optimize = widgets.interact.options(manual=True, manual_name="Optimize this molecule")

@optimize
def optimizing():
    path_to_mols = main(indx=ind)
    global optimized_mols
    optimized_mols = pd.read_csv(path_to_mols+'/all_valid.csv')
    print('#### Optimizing Done! ####')
    
    



interactive(children=(Button(description='Optimize this molecule', style=ButtonStyle()), Output()), _dom_class…

In [79]:
sorted_mols = optimized_mols.sort_values(by='sa_score')
molec = [mol for mol in sorted_mols['generated_coformers']]
sa = [round(mol,2) for mol in sorted_mols['sa_score']]
#fig, ax = plt.subplots(2,5, figsize=(4,4))
rw_molecules =  [Chem.MolFromSmiles(mol) for mol in sorted_mols['generated_coformers']]

img=Draw.MolsToGridImage(rw_molecules,
                                    molsPerRow=min(4, len(rw_molecules)),
                                    legends=[str(mol[0]+'\n  SA score:  '+str(mol[1])) for mol in zip(molec,sa)],
                                    subImgSize=(400, 400),returnPNG=False)
img.save('fig.png')

In [77]:
show_optimized = widgets.interact.options(manual=True, manual_name="Show Molecules")
@show_optimized
def generate():
    sorted_mols = optimized_mols.sort_values(by='sa_score')
    molec = [mol for mol in sorted_mols['generated_coformers']]
    sa = [round(mol,2) for mol in sorted_mols['sa_score']]
    #fig, ax = plt.subplots(2,5, figsize=(4,4))
    rw_molecules =  [Chem.MolFromSmiles(mol) for mol in sorted_mols['generated_coformers']]
    
    display(Draw.MolsToGridImage(rw_molecules,
                                    molsPerRow=min(4, len(rw_molecules)),
                                    legends=[str(mol[0]+'\n  SA score:  '+str(mol[1])) for mol in zip(molec,sa)],
                                    subImgSize=(400, 400)))
    
    # for mol in optimized_mols.sort_values(by='orthogonal_planes')['generated_coformers']:
    #     display(SVG(moltosvg(Chem.MolFromSmiles(mol))))

interactive(children=(Button(description='Show Molecules', style=ButtonStyle()), Output()), _dom_classes=('wid…

In [14]:
# horizon_w = widgets.Dropdown(
#     options=[10, 20, 30, 60],
#     value=30,
#     #60
#     description='Horizon:',
#     disabled=False
# )

# history_w = widgets.Dropdown(
#     options=[50, 100, 200, 250, 300, 500],
#     value=200,
#     #300
#     description='History size:',
#     disabled=False
# )

# timeout_w = widgets.Dropdown(
#     options=[15, 30, 60, 120, 300],
#     value=15,
#     #120
#     description='Timeout (sec)',
#     disabled=False
# )

In [15]:
# display(horizon_w)
# display(history_w)
# display(timeout_w)

In [16]:
# interact_calc=widgets.interact.options(manual=True, manual_name="Calculate")

# @interact_calc
# def process_pipeline():    
#     horizon = horizon_w.value
#     history_size = history_w.value
#     timeout_sec = timeout_w.value
#     timeout = timeout_w.value / 60.
#     print(f"Forecasting horizon: {horizon}")
#     print(f"History size: {history_size}")
#     print(f"Timeout: {timeout} min")
    
#     if horizon * 2 > history_size:
#         print("History size is too low! It has to be at least x2 horizon!")

#     with open(f"data/63/{horizon}_{history_size}_{timeout_sec}_forecast.pickle", "rb") as f:
#         forecast = pickle.load(f)
    
#     with open(f"data/63/{horizon}_{history_size}_{timeout_sec}_pipeline.pickle", "rb") as p:
#         pipeline = pickle.load(p)
    
#     print("Pipeline Graph:")
    
#     pipeline.show()

#     print("Forecasting:")
    
#     plot_automl_forecast(df, forecast_horizon=horizon, history_size=history_size, forecast=forecast)